In [18]:
import cv2
import math
import random
import numpy as np
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", DEVICE)

# =====================
# PATHS
# =====================
DATA_ROOT = Path("../data/datasets")

RAILSEM_DIR = Path("../data/datasets/RailSem19/jpgs/rs19_val")

BDD_DIR     = DATA_ROOT / "BDD100K/night_frames"

OUT_DIR = Path("../outputs/models/deblur")
OUT_DIR.mkdir(parents=True, exist_ok=True)

LOG_DIR = Path("../outputs/logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)

# =====================
# TRAIN PARAMS
# =====================
IMG_SIZE = 256
BATCH_SIZE = 4
LR = 1e-4

EPOCHS_STAGE1 = 60   # RailSem
EPOCHS_STAGE2 = 40   # BDD Night


[INFO] Using device: cuda


In [19]:
def motion_blur_kernel(size, angle):
    kernel = np.zeros((size, size))
    kernel[size // 2, :] = np.ones(size)
    M = cv2.getRotationMatrix2D((size//2, size//2), angle, 1)
    kernel = cv2.warpAffine(kernel, M, (size, size))
    kernel /= kernel.sum()
    return kernel

def apply_motion_blur(img):
    k = random.choice([7, 9, 11, 15])
    angle = random.uniform(-15, 15)
    kernel = motion_blur_kernel(k, angle)
    return cv2.filter2D(img, -1, kernel)


In [20]:
class DeblurDataset(Dataset):
    def __init__(self, img_dir):
        self.images = list(img_dir.glob("*.jpg"))

        if len(self.images) == 0:
            raise RuntimeError(f"No images found in {img_dir}")

        print(f"[INFO] Loaded {len(self.images)} images from {img_dir}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = cv2.imread(str(self.images[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        sharp = img.astype(np.float32) / 255.0
        blur  = apply_motion_blur(img).astype(np.float32) / 255.0

        return (
            torch.from_numpy(blur).permute(2,0,1),
            torch.from_numpy(sharp).permute(2,0,1)
        )


In [21]:
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, 3, padding=1)
        self.conv2 = nn.Conv2d(c, c, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))

class DeblurNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.head = nn.Conv2d(3, 64, 3, padding=1)
        self.body = nn.Sequential(*[ResBlock(64) for _ in range(12)])
        self.tail = nn.Conv2d(64, 3, 3, padding=1)

    def forward(self, x):
        x = torch.relu(self.head(x))
        x = self.body(x)
        return torch.clamp(self.tail(x), 0, 1)


In [22]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    psnr_vals, ssim_vals = [], []

    for blur, sharp in loader:
        blur, sharp = blur.to(DEVICE), sharp.to(DEVICE)
        out = model(blur)

        out_np = out[0].permute(1,2,0).cpu().numpy()
        gt_np  = sharp[0].permute(1,2,0).cpu().numpy()

        psnr_vals.append(psnr(gt_np, out_np, data_range=1.0))
        ssim_vals.append(
            ssim(gt_np, out_np, channel_axis=2, data_range=1.0)
        )

    return np.mean(psnr_vals), np.mean(ssim_vals)


In [23]:
def save_checkpoint(model, optimizer, epoch, best_psnr, path):
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_psnr": best_psnr
    }, path)

def save_epoch(model, epoch, tag):
    torch.save(
        model.state_dict(),
        OUT_DIR / f"{tag}_epoch_{epoch:03d}.pth"
    )


In [24]:
def train_stage(
    model, train_loader, val_loader,
    optimizer, start_epoch, epochs, tag
):
    best_psnr = -1

    for epoch in range(start_epoch, start_epoch + epochs):
        model.train()
        total_loss = 0

        for blur, sharp in tqdm(train_loader, desc=f"{tag} Epoch {epoch}"):
            blur, sharp = blur.to(DEVICE), sharp.to(DEVICE)

            optimizer.zero_grad()
            out = model(blur)
            loss = nn.L1Loss()(out, sharp)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        val_psnr, val_ssim = evaluate(model, val_loader)

        print(
            f"[{tag}] Epoch {epoch} | "
            f"Loss={avg_loss:.4f} | "
            f"PSNR={val_psnr:.2f} | "
            f"SSIM={val_ssim:.4f}"
        )

        save_epoch(model, epoch, tag)

        if val_psnr > best_psnr:
            best_psnr = val_psnr
            torch.save(
                model.state_dict(),
                OUT_DIR / f"best_{tag}.pth"
            )

        save_checkpoint(
            model, optimizer, epoch, best_psnr,
            OUT_DIR / f"{tag}_checkpoint.pth"
        )


In [ ]:
rail_train = DeblurDataset(RAILSEM_DIR)

rail_loader = DataLoader(
    rail_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
rail_val   = DeblurDataset(RAILSEM_DIR)
rail_val_loader = DataLoader(
    rail_val, batch_size=1,
    shuffle=False, num_workers=2
)

model = DeblurNet().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)

train_stage(
    model,
    rail_loader,
    rail_val_loader,
    optimizer,
    start_epoch=0,
    epochs=EPOCHS_STAGE1,
    tag="railsem"
)


[INFO] Loaded 8500 images from ..\data\datasets\RailSem19\jpgs\rs19_val
[INFO] Loaded 8500 images from ..\data\datasets\RailSem19\jpgs\rs19_val


railsem Epoch 0:   0%|          | 0/2125 [00:00<?, ?it/s]

In [ ]:
bdd_train = DeblurDataset(BDD_DIR)
bdd_val   = DeblurDataset(BDD_DIR)

bdd_loader = DataLoader(
    bdd_train, batch_size=BATCH_SIZE,
    shuffle=True, num_workers=4, pin_memory=True
)

bdd_val_loader = DataLoader(
    bdd_val, batch_size=1,
    shuffle=False, num_workers=2
)

optimizer = optim.Adam(model.parameters(), lr=LR * 0.5)

train_stage(
    model,
    bdd_loader,
    bdd_val_loader,
    optimizer,
    start_epoch=EPOCHS_STAGE1,
    epochs=EPOCHS_STAGE2,
    tag="bdd_night"
)

torch.save(
    model.state_dict(),
    OUT_DIR / "best_model_deblur.pth"
)

print("✅ Final deblur model saved")


AssertionError: No images found in ..\data\datasets\BDD100K\night_frames

In [ ]:
# ============================================================
# SINGLE CELL – COMPLETE DEBLUR TRAINING WITH PSNR & SSIM
# ============================================================

import cv2, torch, random, numpy as np
from pathlib import Path
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# ---------------- CONFIG ----------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", DEVICE)

DATA_ROOT = Path("../data/datasets/RailSem19")
TRAIN_DIR = DATA_ROOT / "images_train"
VAL_DIR   = DATA_ROOT / "images_val"

OUT_DIR = Path("../outputs/models/deblur")
OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 50

# ---------------- MOTION BLUR ----------------
def motion_blur_kernel(size, angle):
    k = np.zeros((size, size))
    k[size // 2] = np.ones(size)
    M = cv2.getRotationMatrix2D((size//2, size//2), angle, 1)
    k = cv2.warpAffine(k, M, (size, size))
    return k / k.sum()

def apply_motion_blur(img):
    ksize = random.choice([7, 9, 11, 15])
    angle = random.uniform(-15, 15)
    return cv2.filter2D(img, -1, motion_blur_kernel(ksize, angle))

# ---------------- DATASET ----------------
class DeblurDataset(Dataset):
    def __init__(self, folder):
        self.files = list(folder.glob("*.jpg")) + list(folder.glob("*.png"))
        assert len(self.files) > 0, f"No images in {folder}"

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        img = cv2.imread(str(self.files[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        sharp = img.astype(np.float32) / 255.0
        blur  = apply_motion_blur(img).astype(np.float32) / 255.0

        return (
            torch.from_numpy(blur).permute(2,0,1),
            torch.from_numpy(sharp).permute(2,0,1)
        )

# ---------------- MODEL ----------------
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.c1 = nn.Conv2d(c, c, 3, padding=1)
        self.c2 = nn.Conv2d(c, c, 3, padding=1)
        self.r  = nn.ReLU(inplace=True)
    def forward(self, x):
        return x + self.c2(self.r(self.c1(x)))

class DeblurNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.h = nn.Conv2d(3, 64, 3, padding=1)
        self.b = nn.Sequential(*[ResBlock(64) for _ in range(12)])
        self.t = nn.Conv2d(64, 3, 3, padding=1)
    def forward(self, x):
        return torch.clamp(self.t(self.b(torch.relu(self.h(x)))), 0, 1)

# ---------------- METRICS ----------------
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    P, S = [], []
    for b, s in loader:
        b, s = b.to(DEVICE), s.to(DEVICE)
        o = model(b)[0].permute(1,2,0).cpu().numpy()
        s = s[0].permute(1,2,0).cpu().numpy()
        P.append(psnr(s, o, data_range=1.0))
        S.append(ssim(s, o, channel_axis=2, data_range=1.0))
    return float(np.mean(P)), float(np.mean(S))

# ---------------- LOADERS ----------------
train_ds = DeblurDataset(TRAIN_DIR)
val_ds   = DeblurDataset(VAL_DIR)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

# ---------------- TRAIN ----------------
model = DeblurNet().to(DEVICE)
opt = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.L1Loss()
best_psnr = 0.0

for epoch in range(EPOCHS):
    model.train()
    loss_sum = 0

    for blur, sharp in tqdm(train_loader, desc=f"Epoch {epoch}"):
        blur, sharp = blur.to(DEVICE), sharp.to(DEVICE)
        opt.zero_grad()
        out = model(blur)
        loss = loss_fn(out, sharp)
        loss.backward()
        opt.step()
        loss_sum += loss.item()

    loss_avg = loss_sum / len(train_loader)
    v_psnr, v_ssim = evaluate(model, val_loader)

    print(
        f"[Epoch {epoch}] "
        f"TrainLoss={loss_avg:.4f} | "
        f"PSNR={v_psnr:.2f} dB | "
        f"SSIM={v_ssim:.4f}"
    )

    torch.save(
        {"epoch": epoch, "model": model.state_dict(),
         "loss": loss_avg, "psnr": v_psnr, "ssim": v_ssim},
        OUT_DIR / f"epoch_{epoch}.pth"
    )

    if v_psnr > best_psnr:
        best_psnr = v_psnr
        torch.save({"model": model.state_dict()},
                   OUT_DIR / "best_model_deblur.pth")

print("✅ Training complete")


[INFO] Using device: cuda


AssertionError: No images in ..\data\datasets\RailSem19\images_train

In [3]:
# ============================================================
# RAILSEM19 DEBLUR TRAINING – CORRECT STRUCTURE – ONE CELL
# ============================================================

import cv2, torch, random, numpy as np
from pathlib import Path
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# ---------------- CONFIG ----------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", DEVICE)

DATA_ROOT = Path("../data/datasets/RailSem19/jpgs")
TRAIN_DIR = DATA_ROOT / "rs19"
VAL_DIR   = DATA_ROOT / "rs19_val"

OUT_DIR = Path("../outputs/models/deblur")
OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 40

# ---------------- MOTION BLUR ----------------
def motion_blur_kernel(size, angle):
    kernel = np.zeros((size, size))
    kernel[size // 2] = np.ones(size)
    M = cv2.getRotationMatrix2D((size//2, size//2), angle, 1)
    kernel = cv2.warpAffine(kernel, M, (size, size))
    return kernel / kernel.sum()

def apply_motion_blur(img):
    k = random.choice([9, 11, 15])
    a = random.uniform(-20, 20)
    return cv2.filter2D(img, -1, motion_blur_kernel(k, a))

# ---------------- DATASET ----------------
class RailSemDeblurDataset(Dataset):
    def __init__(self, folder):
        self.files = sorted(list(folder.glob("*.jpg")))
        assert len(self.files) > 0, f"No images found in {folder}"

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = cv2.imread(str(self.files[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        sharp = img.astype(np.float32) / 255.0
        blur  = apply_motion_blur(img).astype(np.float32) / 255.0

        return (
            torch.from_numpy(blur).permute(2,0,1),
            torch.from_numpy(sharp).permute(2,0,1)
        )

# ---------------- MODEL ----------------
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.c1 = nn.Conv2d(c, c, 3, padding=1)
        self.c2 = nn.Conv2d(c, c, 3, padding=1)
        self.r  = nn.ReLU(inplace=True)
    def forward(self, x):
        return x + self.c2(self.r(self.c1(x)))

class DeblurNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.h = nn.Conv2d(3, 64, 3, padding=1)
        self.b = nn.Sequential(*[ResBlock(64) for _ in range(12)])
        self.t = nn.Conv2d(64, 3, 3, padding=1)
    def forward(self, x):
        return torch.clamp(self.t(self.b(torch.relu(self.h(x)))), 0, 1)

# ---------------- METRICS ----------------
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    P, S = [], []
    for b, s in loader:
        b, s = b.to(DEVICE), s.to(DEVICE)
        o = model(b)[0].permute(1,2,0).cpu().numpy()
        s = s[0].permute(1,2,0).cpu().numpy()
        P.append(psnr(s, o, data_range=1.0))
        S.append(ssim(s, o, channel_axis=2, data_range=1.0))
    return float(np.mean(P)), float(np.mean(S))

# ---------------- LOADERS ----------------
train_ds = RailSemDeblurDataset(TRAIN_DIR)
val_ds   = RailSemDeblurDataset(VAL_DIR)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

print(f"[INFO] Train images: {len(train_ds)}")
print(f"[INFO] Val images:   {len(val_ds)}")

# ---------------- TRAIN ----------------
model = DeblurNet().to(DEVICE)
opt = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.L1Loss()
best_psnr = 0.0

for epoch in range(EPOCHS):
    model.train()
    loss_sum = 0

    for blur, sharp in tqdm(train_loader, desc=f"Epoch {epoch}"):
        blur, sharp = blur.to(DEVICE), sharp.to(DEVICE)
        opt.zero_grad()
        out = model(blur)
        loss = loss_fn(out, sharp)
        loss.backward()
        opt.step()
        loss_sum += loss.item()

    loss_avg = loss_sum / len(train_loader)
    v_psnr, v_ssim = evaluate(model, val_loader)

    print(
        f"[Epoch {epoch}] "
        f"TrainLoss={loss_avg:.4f} | "
        f"PSNR={v_psnr:.2f} dB | "
        f"SSIM={v_ssim:.4f}"
    )

    torch.save(
        {"epoch": epoch, "model": model.state_dict(),
         "loss": loss_avg, "psnr": v_psnr, "ssim": v_ssim},
        OUT_DIR / f"epoch_{epoch}.pth"
    )

    if v_psnr > best_psnr:
        best_psnr = v_psnr
        torch.save({"model": model.state_dict()},
                   OUT_DIR / "best_model_deblur.pth")

print("✅ RailSem19 Deblur training finished")


[INFO] Using device: cuda


AssertionError: No images found in ..\data\datasets\RailSem19\jpgs\rs19

In [ ]:
# ============================================================
# RAILSEM19 DEBLUR TRAINING – AUTO SPLIT (YOUR STRUCTURE)
# ============================================================

import cv2, torch, random, numpy as np
from pathlib import Path
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# ---------------- CONFIG ----------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", DEVICE)

DATA_DIR = Path("../data/datasets/RailSem19/jpgs/rs19_val")
assert DATA_DIR.exists(), f"Folder missing: {DATA_DIR}"

OUT_DIR = Path("../outputs/models/deblur")
OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 40
TRAIN_SPLIT = 0.9

# ---------------- MOTION BLUR ----------------
def motion_blur_kernel(size, angle):
    k = np.zeros((size, size))
    k[size//2] = np.ones(size)
    M = cv2.getRotationMatrix2D((size//2, size//2), angle, 1)
    k = cv2.warpAffine(k, M, (size, size))
    return k / k.sum()

def apply_motion_blur(img):
    k = random.choice([9, 11, 15])
    a = random.uniform(-20, 20)
    return cv2.filter2D(img, -1, motion_blur_kernel(k, a))

# ---------------- DATASET ----------------
class RailSemDeblurDataset(Dataset):
    def __init__(self, folder):
        self.files = sorted(list(folder.glob("*.jpg")))
        assert len(self.files) > 0, f"No images found in {folder}"

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = cv2.imread(str(self.files[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        sharp = img.astype(np.float32) / 255.0
        blur  = apply_motion_blur(img).astype(np.float32) / 255.0

        return (
            torch.from_numpy(blur).permute(2,0,1),
            torch.from_numpy(sharp).permute(2,0,1)
        )

# ---------------- MODEL ----------------
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.c1 = nn.Conv2d(c, c, 3, padding=1)
        self.c2 = nn.Conv2d(c, c, 3, padding=1)
        self.r  = nn.ReLU(inplace=True)
    def forward(self, x):
        return x + self.c2(self.r(self.c1(x)))

class DeblurNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.h = nn.Conv2d(3, 64, 3, padding=1)
        self.b = nn.Sequential(*[ResBlock(64) for _ in range(12)])
        self.t = nn.Conv2d(64, 3, 3, padding=1)
    def forward(self, x):
        return torch.clamp(self.t(self.b(torch.relu(self.h(x)))), 0, 1)

# ---------------- METRICS ----------------
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    P, S = [], []
    for b, s in loader:
        b, s = b.to(DEVICE), s.to(DEVICE)
        o = model(b)[0].permute(1,2,0).cpu().numpy()
        s = s[0].permute(1,2,0).cpu().numpy()
        P.append(psnr(s, o, data_range=1.0))
        S.append(ssim(s, o, channel_axis=2, data_range=1.0))
    return float(np.mean(P)), float(np.mean(S))

# ---------------- SPLIT ----------------
full_ds = RailSemDeblurDataset(DATA_DIR)
n_train = int(len(full_ds) * TRAIN_SPLIT)
n_val   = len(full_ds) - n_train
train_ds, val_ds = random_split(full_ds, [n_train, n_val])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

print(f"[INFO] Total images: {len(full_ds)}")
print(f"[INFO] Train images: {len(train_ds)}")
print(f"[INFO] Val images:   {len(val_ds)}")

# ---------------- TRAIN ----------------
model = DeblurNet().to(DEVICE)
opt = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.L1Loss()
best_psnr = 0.0

for epoch in range(EPOCHS):
    model.train()
    loss_sum = 0

    for blur, sharp in tqdm(train_loader, desc=f"Epoch {epoch}"):
        blur, sharp = blur.to(DEVICE), sharp.to(DEVICE)
        opt.zero_grad()
        out = model(blur)
        loss = loss_fn(out, sharp)
        loss.backward()
        opt.step()
        loss_sum += loss.item()

    loss_avg = loss_sum / len(train_loader)
    v_psnr, v_ssim = evaluate(model, val_loader)

    print(
        f"[Epoch {epoch}] "
        f"TrainLoss={loss_avg:.4f} | "
        f"PSNR={v_psnr:.2f} dB | "
        f"SSIM={v_ssim:.4f}"
    )

    torch.save(
        {"epoch": epoch, "model": model.state_dict(),
         "loss": loss_avg, "psnr": v_psnr, "ssim": v_ssim},
        OUT_DIR / f"epoch_{epoch}.pth"
    )

    if v_psnr > best_psnr:
        best_psnr = v_psnr
        torch.save({"model": model.state_dict()},
                   OUT_DIR / "best_model_deblur.pth")

print("✅ RailSem19 Deblur training finished")


[INFO] Using device: cuda
[INFO] Total images: 8500
[INFO] Train images: 7650
[INFO] Val images:   850


Epoch 0:   0%|          | 0/1913 [00:00<?, ?it/s]

In [4]:
# ============================================================
# RAILSEM19 DEBLUR TRAINING – FIXED & MORE STABLE VERSION
# ============================================================

import cv2
import cv2
import torch
import random
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch.nn as nn               # ← fixed
import torch.optim as optim         # ← use this instead of aliasing to nn
from torch.utils.data import Dataset, DataLoader, random_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# ────────────────────────────────────────────────
#  CONFIG
# ────────────────────────────────────────────────

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {DEVICE}")

DATA_DIR = Path("../data/datasets/RailSem19/jpgs/rs19_val")
assert DATA_DIR.exists(), f"Folder not found: {DATA_DIR}"

OUT_DIR = Path("../outputs/models/deblur")
OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE    = 256
BATCH_SIZE  = 1          # start small → increase later if stable
LR          = 1e-4
EPOCHS      = 100
TRAIN_SPLIT = 0.9

NUM_WORKERS    = 0       # 0 = safest, increase to 2–4 later if needed
PIN_MEMORY     = torch.cuda.is_available()

# ────────────────────────────────────────────────
#  MOTION BLUR KERNEL
# ────────────────────────────────────────────────

def motion_blur_kernel(size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[size//2, :] = 1.0
    M = cv2.getRotationMatrix2D((size//2, size//2), angle, 1.0)
    k = cv2.warpAffine(k, M, (size, size), flags=cv2.INTER_LINEAR)
    return k / k.sum()

def apply_motion_blur(img):
    try:
        kernel_size = random.choice([9, 11, 13, 15])
        angle = random.uniform(-35, 35)
        kernel = motion_blur_kernel(kernel_size, angle)
        blurred = cv2.filter2D(img, -1, kernel)
        return blurred
    except Exception as e:
        print(f"Blur failed: {e}")
        return img  # fallback to sharp if blur crashes

# ────────────────────────────────────────────────
#  DATASET
# ────────────────────────────────────────────────

class RailSemDeblurDataset(Dataset):
    def __init__(self, folder: Path):
        self.files = sorted(folder.glob("*.jpg"))
        if not self.files:
            raise ValueError(f"No .jpg files found in {folder}")
        print(f"[DATASET] Found {len(self.files)} images")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        img = cv2.imread(str(path))
        if img is None:
            raise IOError(f"Cannot read image: {path}")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

        sharp = img.astype(np.float32) / 255.0
        blur  = apply_motion_blur(img).astype(np.float32) / 255.0

        return (
            torch.from_numpy(blur ).permute(2, 0, 1),
            torch.from_numpy(sharp).permute(2, 0, 1)
        )

# ────────────────────────────────────────────────
#  MODEL
# ────────────────────────────────────────────────

class ResBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=True)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=True)
        self.relu  = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))

class DeblurNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.head   = nn.Conv2d(3,   64, 3, padding=1)
        self.body   = nn.Sequential(*[ResBlock(64) for _ in range(12)])
        self.tail   = nn.Conv2d(64,  3,  3, padding=1)

    def forward(self, x):
        feat = torch.relu(self.head(x))
        feat = self.body(feat)
        out  = self.tail(feat)
        return torch.clamp(out, 0.0, 1.0)

# ────────────────────────────────────────────────
#  EVALUATION
# ────────────────────────────────────────────────

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    psnrs, ssims = [], []
    for blur, sharp in loader:
        blur  = blur.to(DEVICE)
        sharp = sharp.to(DEVICE)
        pred = model(blur)

        # take first image in batch for metric
        p = pred[0].permute(1,2,0).cpu().numpy()
        s = sharp[0].permute(1,2,0).cpu().numpy()

        psnrs.append(psnr(s, p, data_range=1.0))
        ssims.append(ssim(s, p, channel_axis=2, data_range=1.0))

    return float(np.mean(psnrs)), float(np.mean(ssims))

# ────────────────────────────────────────────────
#  MAIN
# ────────────────────────────────────────────────

if __name__ == "__main__":
    # ── Dataset + split ────────────────────────────────
    full_dataset = RailSemDeblurDataset(DATA_DIR)

    n_total = len(full_dataset)
    n_train = int(n_total * TRAIN_SPLIT)
    n_val   = n_total - n_train

    train_ds, val_ds = random_split(full_dataset, [n_train, n_val])

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        drop_last=True,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=1,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
    )

    print(f"[INFO] Total: {n_total}  |  Train: {len(train_ds)}  |  Val: {len(val_ds)}")
    print(f"[INFO] Batch size: {BATCH_SIZE}  |  Workers: {NUM_WORKERS}  |  Pin memory: {PIN_MEMORY}")

    # ── Quick test: can we load one batch? ─────────────
    print("\nTesting first batch ... ", end="")
    try:
        test_blur, test_sharp = next(iter(train_loader))
        print("OK")
        print(f"   shapes → blur: {test_blur.shape}   sharp: {test_sharp.shape}")
    except Exception as e:
        print("FAILED")
        print(e)
        exit(1)

    # ── Model + Optimizer + Loss ───────────────────────
    model = DeblurNet().to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn = nn.L1Loss()

    best_psnr = -1.0
    best_path = OUT_DIR / "best_model_deblur.pth"

    # ── Training loop ──────────────────────────────────
    for epoch in range(1, EPOCHS + 1):
        model.train()
        total_loss = 0.0
        batch_count = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
        for blur, sharp in pbar:
            blur  = blur.to(DEVICE)
            sharp = sharp.to(DEVICE)

            optimizer.zero_grad()
            pred = model(blur)
            loss = loss_fn(pred, sharp)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

            pbar.set_postfix(loss=f"{loss.item():.4f}")

            # occasional print
            if batch_count % 100 == 0:
                print(f"  [batch {batch_count:4d}] loss={loss.item():.4f}")

        avg_loss = total_loss / batch_count

        # ── Validation ────────────────────────────────
        val_psnr, val_ssim = evaluate(model, val_loader)

        print(f"[Epoch {epoch:2d}] "
              f"Loss={avg_loss:.4f} | "
              f"PSNR={val_psnr:.2f} dB | "
              f"SSIM={val_ssim:.4f}")

        # Save every epoch
        torch.save(
            {
                "epoch": epoch,
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "loss": avg_loss,
                "val_psnr": val_psnr,
                "val_ssim": val_ssim,
            },
            OUT_DIR / f"epoch_{epoch:03d}.pth"
        )

        # Save best model
        if val_psnr > best_psnr:
            best_psnr = val_psnr
            torch.save(model.state_dict(), best_path)
            print(f"  → New best model saved (PSNR {val_psnr:.2f})")

    print("\n" + "═" * 60)
    print("Training finished. Best PSNR:", f"{best_psnr:.2f}")
    print("Best model saved at:", best_path)

[INFO] Using device: cuda
[DATASET] Found 8500 images
[INFO] Total: 8500  |  Train: 7650  |  Val: 850
[INFO] Batch size: 1  |  Workers: 0  |  Pin memory: True

Testing first batch ... OK
   shapes → blur: torch.Size([1, 3, 256, 256])   sharp: torch.Size([1, 3, 256, 256])


Epoch 1/100:   1%|▏         | 101/7650 [00:10<12:18, 10.22it/s, loss=0.0449]

  [batch  100] loss=0.0433


Epoch 1/100:   3%|▎         | 200/7650 [00:19<10:37, 11.69it/s, loss=0.0360]

  [batch  200] loss=0.0400


...

  [batch 1400] loss=0.0295


Epoch 3/100:  20%|█▉        | 1500/7650 [02:29<12:07,  8.46it/s, loss=0.0349]

  [batch 1500] loss=0.0130


Epoch 3/100:  20%|██        | 1561/7650 [02:35<10:07, 10.03it/s, loss=0.0277]


KeyboardInterrupt: 

In [5]:
# ============================================================
# RAILWAY DEBLUR TRAINING PIPELINE (SYNTHETIC MOTION BLUR)
# ============================================================

import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# =====================
# CONFIG
# =====================
DATA_ROOT = Path("../data/datasets/RailSem19/JPEGImages")
SAVE_DIR  = Path("../outputs/models/deblur")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
BATCH_SIZE = 4
EPOCHS = 30
LR = 1e-4

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Device:", DEVICE)

# =====================
# SYNTHETIC BLUR
# =====================
def apply_motion_blur(img, max_kernel=25):
    k = np.random.randint(7, max_kernel)
    kernel = np.zeros((k, k))
    kernel[k // 2, :] = np.ones(k)
    kernel /= k
    return cv2.filter2D(img, -1, kernel)

def add_night_noise(img):
    noise = np.random.normal(0, 8, img.shape)
    noisy = img + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

# =====================
# DATASET
# =====================
class RailBlurDataset(Dataset):
    def __init__(self, root):
        self.images = list(root.glob("*.jpg"))
        assert len(self.images) > 0, "No images found!"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = cv2.imread(str(self.images[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        sharp = img.copy()
        blur  = apply_motion_blur(img)
        blur  = add_night_noise(blur)

        sharp = torch.from_numpy(sharp / 255.0).permute(2,0,1).float()
        blur  = torch.from_numpy(blur  / 255.0).permute(2,0,1).float()

        return blur, sharp

dataset = RailBlurDataset(DATA_ROOT)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

print(f"[INFO] Training samples: {len(dataset)}")

# =====================
# MODEL
# =====================
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, 3, padding=1)
        self.conv2 = nn.Conv2d(c, c, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))

class DeblurGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.head = nn.Conv2d(3, 64, 7, padding=3)
        self.body = nn.Sequential(*[ResBlock(64) for _ in range(9)])
        self.tail = nn.Conv2d(64, 3, 7, padding=3)

    def forward(self, x):
        x = torch.relu(self.head(x))
        x = self.body(x)
        return torch.sigmoid(self.tail(x))

model = DeblurGenerator().to(DEVICE)

# =====================
# TRAINING SETUP
# =====================
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LR)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))

# =====================
# TRAIN LOOP
# =====================
print("[INFO] Starting training...")

best_loss = 1e9

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for blur, sharp in tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        blur  = blur.to(DEVICE)
        sharp = sharp.to(DEVICE)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=(DEVICE.type == "cuda")):
            pred = model(blur)
            loss = criterion(pred, sharp)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    epoch_loss /= len(loader)
    print(f"[Epoch {epoch+1}] L1 Loss: {epoch_loss:.4f}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(
            {"model_state": model.state_dict()},
            SAVE_DIR / "best_model_deblur.pth"
        )
        print("✅ Saved best model")

print("🎉 TRAINING COMPLETE")

[INFO] Device: cuda


AssertionError: No images found!

In [1]:
# ============================================================
# RAILWAY DEBLUR TRAINING (RailSem19 rs19_val)
# ============================================================

import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random

# =====================
# CONFIG
# =====================
DATA_DIR = Path("../data/datasets/RailSem19/jpgs/rs19_val")
SAVE_DIR = Path("../outputs/models/deblur")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
BATCH_SIZE = 4
EPOCHS = 30
LR = 1e-4

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Device:", DEVICE)

# =====================
# SYNTHETIC BLUR (PROPER)
# =====================
def motion_blur(img):
    k = random.choice([9, 11, 15, 21])
    angle = random.uniform(-20, 20)  # train motion variation

    kernel = np.zeros((k, k))
    kernel[k // 2, :] = np.ones(k)
    kernel /= k

    M = cv2.getRotationMatrix2D((k//2, k//2), angle, 1)
    kernel = cv2.warpAffine(kernel, M, (k, k))

    return cv2.filter2D(img, -1, kernel)

def night_noise(img):
    noise = np.random.normal(0, 10, img.shape)
    noisy = img + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

# =====================
# DATASET (FIXED)
# =====================
class RailSemBlurDataset(Dataset):
    def __init__(self, root):
        self.images = list(root.rglob("*.jpg"))
        assert len(self.images) > 0, "❌ No images found"
        print(f"[INFO] Loaded {len(self.images)} images")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = cv2.imread(str(self.images[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        sharp = img.copy()
        blur = motion_blur(img)
        blur = night_noise(blur)

        sharp = torch.from_numpy(sharp / 255.0).permute(2,0,1).float()
        blur  = torch.from_numpy(blur  / 255.0).permute(2,0,1).float()

        return blur, sharp

dataset = RailSemBlurDataset(DATA_DIR)

loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,   # 🔥 CRITICAL FIX
    pin_memory=True
)


# =====================
# MODEL
# =====================
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, 3, padding=1)
        self.conv2 = nn.Conv2d(c, c, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))

class DeblurGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.head = nn.Conv2d(3, 64, 7, padding=3)
        self.body = nn.Sequential(*[ResBlock(64) for _ in range(9)])
        self.tail = nn.Conv2d(64, 3, 7, padding=3)

    def forward(self, x):
        x = torch.relu(self.head(x))
        x = self.body(x)
        return torch.sigmoid(self.tail(x))

model = DeblurGenerator().to(DEVICE)

# =====================
# TRAINING
# =====================
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))

best_loss = float("inf")
print("[INFO] Training started")

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for blur, sharp in tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        blur = blur.to(DEVICE)
        sharp = sharp.to(DEVICE)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=(DEVICE.type == "cuda")):
            pred = model(blur)
            loss = criterion(pred, sharp)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    epoch_loss /= len(loader)
    print(f"[Epoch {epoch+1}] L1 Loss: {epoch_loss:.4f}")

    torch.save(
        {
            "epoch": epoch,
            "loss": epoch_loss,
            "model_state": model.state_dict()
        },
        SAVE_DIR / f"epoch_{epoch+1}.pth"
    )

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(
            {"model_state": model.state_dict()},
            SAVE_DIR / "best_model_deblur.pth"
        )
        print("✅ Best model updated")

print("🎉 TRAINING COMPLETE")


[INFO] Device: cuda
[INFO] Loaded 8500 images


C:\Users\Swayam\AppData\Local\Temp\ipykernel_18764\3121918670.py:120: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))


[INFO] Training started


Epoch 1/30:   0%|          | 0/2125 [00:00<?, ?it/s]C:\Users\Swayam\AppData\Local\Temp\ipykernel_18764\3121918670.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type == "cuda")):
Epoch 1/30: 100%|██████████| 2125/2125 [10:16<00:00,  3.45it/s]


[Epoch 1] L1 Loss: 0.0505
✅ Best model updated


...

[Epoch 26] L1 Loss: 0.0384
✅ Best model updated


Epoch 27/30: 100%|██████████| 2125/2125 [14:18<00:00,  2.47it/s]


[Epoch 27] L1 Loss: 0.0383
✅ Best model updated


Epoch 28/30:  26%|██▌       | 553/2125 [04:05<11:38,  2.25it/s]


KeyboardInterrupt: 